In [8]:
import os
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))

import time
import glob
from pathlib import Path

import rasterio
from rasterio.enums import Resampling

In [16]:
# even number only
upscale_factor = 4.2

original_x = 0.3125
original_y = 0.25

desired_scale = 0.045

width_upscale_factor = original_x / desired_scale
height_upscale_factor = original_y / desired_scale

print(width_upscale_factor)
print(height_upscale_factor)

6.944444444444445
5.555555555555555


In [10]:
model_out = f'{BASE_DIR}/forecast_data_thailand/'
model_out

'/Users/biplovbhandari/UAH/GRA_Work/AQ_Downscaling/forecast_data_thailand/'

In [11]:
out_path = f'{BASE_DIR}/model_upscaled_3/'
Path(f'{out_path}').mkdir(parents=True, exist_ok=True)
out_path

'/Users/biplovbhandari/UAH/GRA_Work/AQ_Downscaling/model_upscaled_3/'

In [12]:
dates = [f.path.split(model_out)[1] for f in os.scandir(model_out) if f.is_dir()]

In [13]:
start = time.time()

completed = 0

for index, date in enumerate(dates):
    tifs = glob.glob(f'{model_out}{date}/*.tif')
    for tif in tifs:
        dataset = rasterio.open(tif)

        data = dataset.read(
            out_shape = (
                dataset.count,
#                 int(dataset.height * upscale_factor),
#                 int(dataset.width * upscale_factor),
                int(dataset.height * height_upscale_factor),
                int(dataset.width * width_upscale_factor),
            ),
#             resampling = Resampling.bilinear,
            resampling = Resampling.nearest,
        )
        
#         print(f'before:\n {dataset.transform}')

        transform = dataset.transform * dataset.transform.scale(
            (dataset.width / data.shape[-1]),
            (dataset.height / data.shape[-2]),
        )

#         print(f'after:\n {transform}')

        profile = dataset.profile
        profile.update(transform=transform,
#                        width=profile.get('width')*upscale_factor,
#                        height=profile.get('height')*upscale_factor,
                       width=profile.get('width')*width_upscale_factor,
                       height=profile.get('height')*height_upscale_factor,
                      )

        Path(f'{out_path}{date}').mkdir(parents=True, exist_ok=True)

        name = tif.split(f'{model_out}{date}/')[1]
        with rasterio.open(f'{out_path}{date}/{name}', 'w', **profile) as dst:
            dst.write(data)

    completed += 1
    if index % 20 == 0:
        print(f'Completed {completed} out of {len(dates)}: {round(completed / len(dates) * 100, 2)}%')


print(f'Time: {(time.time() - start) / 60.} minutes')

Completed 1 out of 537: 0.19%
Completed 21 out of 537: 3.91%
Completed 41 out of 537: 7.64%
Completed 61 out of 537: 11.36%
Completed 81 out of 537: 15.08%
Completed 101 out of 537: 18.81%
Completed 121 out of 537: 22.53%
Completed 141 out of 537: 26.26%
Completed 161 out of 537: 29.98%
Completed 181 out of 537: 33.71%
Completed 201 out of 537: 37.43%
Completed 221 out of 537: 41.15%
Completed 241 out of 537: 44.88%
Completed 261 out of 537: 48.6%
Completed 281 out of 537: 52.33%
Completed 301 out of 537: 56.05%
Completed 321 out of 537: 59.78%
Completed 341 out of 537: 63.5%
Completed 361 out of 537: 67.23%
Completed 381 out of 537: 70.95%
Completed 401 out of 537: 74.67%
Completed 421 out of 537: 78.4%
Completed 441 out of 537: 82.12%
Completed 461 out of 537: 85.85%
Completed 481 out of 537: 89.57%
Completed 501 out of 537: 93.3%
Completed 521 out of 537: 97.02%
Time: 0.41993560393651325 minutes
